In [1]:
import os

In [2]:
pwd

'E:\\Python practise\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'E:\\Python practise\\Text-Summarizer'

In [11]:
#copied in entity/__init__

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#copied in src/textsummarizer/config/configuration.py

from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [12]:
#copied in src/textsummarizer/config/configuration.py


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [13]:
#copied at components/data_ingestion

import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [21]:
#copied at components/data_ingestion

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download the following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
#copied to pipeline/stage01/dataingestion.py
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-07 12:24:31,267: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-07 12:24:31,271: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-07 12:24:31,274: INFO: common: created directory at: artifacts]
[2023-09-07 12:24:31,277: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-07 12:24:31,280: INFO: 2875715338: File already exist of size: ~ 7718 KB]
